# FBiH Merge Processed Data

In [11]:
import sqlite3

import pandas as pd

from tools.config import SAVE_DIR

## Non political data

**Some Notes on the tables**

`gross_average_wages.xlsx` and `employees.xlsx` both have the same amount of records and go from 2005 to 2024.

`census_2013.xlsx` is only for the year 2013 which is fine as it will be supplemented with political data and be used as a sanity check for making the assumption that people vote based on ethnic lines.

`businesses_per_sector.xlsx` only goes form 2012 to 2023. It might be useful as a proxy for education given the amount of businesses in extractive sectors such as agriculture and/or mining.

Loading all the tables up

In [2]:
gaw = pd.read_excel(SAVE_DIR + "gross_average_wages.xlsx")
registar = pd.read_excel(SAVE_DIR + "businesses_per_sector.xlsx")
census = pd.read_excel(SAVE_DIR + "census_2013.xlsx")
employees = pd.read_excel(SAVE_DIR + "employees.xlsx")

I will do left joins to preserve rows that may not match up perfectly with other tables.

In [3]:
gaw_reg = pd.merge(gaw, registar, on=["Municipality", "Year"], how="left")
gaw_reg_employees = pd.merge(gaw_reg, employees, on=["Municipality", "Year"], how="left")
df = pd.merge(gaw_reg_employees, census, on=["Municipality", "Year"], how="left")

So the data is consistent with political data, I will be fixing the character errors in the municipality names as well as standardizing the names as they were recorded in the political database.

In [4]:
mangled_to_sql = {
    'banovici': 'banovici',
    'bosanska krupa': 'bosanska krupa',
    'bosanski petrovac': 'bosanski petrovac',
    'bosansko grahovo': 'bosansko grahovo',
    'breza': 'breza',
    'bugojno': 'bugojno',
    'busovaca': 'busovaca',
    'bu¸im': 'buzim',
    'capljina': 'capljina',
    'cazin': 'cazin',
    'celic': 'celic',
    'citluk': 'citluk',
    'doboj-istok': 'doboj istok',
    'doboj-jug': 'doboj jug',
    'dobretici': 'dobretici',
    'domaljevac-¦amac': 'domaljevac samac',
    'donji vakuf': 'donji vakuf',
    'drvar': 'drvar',
    'foca': 'foca',
    'fojnica': 'fojnica',
    'glamoc': 'glamoc',
    'gora¸de': 'gorazde',
    'gornji vakuf - uskoplje': 'gornji vakuf uskoplje',
    'gracanica': 'gracanica',
    'grad bihac': 'bihac',
    'grad mostar': 'grad mostar',
    'grad tuzla': 'tuzla',
    'grad zenica': 'zenica',
    'grad ¦iroki brijeg': 'siroki brijeg',
    'gradacac': 'gradacac',
    'grude': 'grude',
    'had¸ici': 'hadzici',
    'ilid¸a': 'ilidza',
    'ilija¨': 'ilijas',
    'jablanica': 'jablanica',
    'jajce': 'jajce',
    'kakanj': 'kakanj',
    'kalesija': 'kalesija',
    'kiseljak': 'kiseljak',
    'kladanj': 'kladanj',
    'kljuc': 'kljuc',
    'konjic': 'konjic',
    'kre¨evo': 'kresevo',
    'kupres': 'kupres',
    'livno': 'livno',
    'ljubu¨ki': 'ljubuski',
    'lukavac': 'lukavac',
    'maglaj': 'maglaj',
    'neum': 'neum',
    'novi grad sarajevo': 'novi grad sarajevo',
    'novi travnik': 'novi travnik',
    'novo sarajevo': 'novo sarajevo',
    'od¸ak': 'odzak',
    'olovo': 'olovo',
    'ora¨je': 'orasje',
    'pale': 'pale',
    'posu¨je': 'posusje',
    'prozor': 'prozor rama',
    'ravno': 'ravno',
    'sanski most': 'sanski most',
    'sapna': 'sapna',
    'sarajevo - centar': 'centar sarajevo',
    'srebrenik': 'srebrenik',
    'stari grad sarajevo': 'stari grad sarajevo',
    'stolac': 'stolac',
    'teocak': 'teocak',
    'te¨anj': 'tesanj',
    'tomislavgrad': 'tomislavgrad',
    'travnik': 'travnik',
    'trnovo': 'trnovo',
    'usora': 'usora',
    'vare¨': 'vares',
    'velika kladu¨a': 'velika kladusa',
    'visoko': 'visoko',
    'vitez': 'vitez',
    'vogo¨ca': 'vogosca',
    'zavidovici': 'zavidovici',
    '´epce': 'zepce',
    '´ivinice': 'zivinice'
}

To add the renamed column.

In [5]:
df["rename"] = df["Municipality"].str.lower().map(mangled_to_sql)

In [6]:
df = df.drop(columns=["Municipality"])
df = df.rename(
    columns={"rename" : "Municipality"}
)

In [7]:
cols = df.columns.tolist()[:-1]
df = df[["Municipality"] + cols]

In [8]:
df

,Municipality,Year,Gross Average Wage,00-Unclassified according to activities CEA 1),"A-Agriculture, forestry and fishing",Activities of households as employers; undifferentiated goods and services-producing activities of households for own use,B-Minning and quarrying,C-Manufacturing,"D-Electricity, gas, steam and air conditioning supply","E-Water supply, sewerage, waste management and remediation activities",...,O-Public administration and defence; compulsory social security,P-Education,Q-Human health and social work activities,"R-Arts, entertainment and recreation",S-Other service activities,Total,Employees,Bosniak,Croat,Serb
0,bihac,2005,824.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11185.0,NaN,NaN,NaN
1,bosanska krupa,2005,717.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2272.0,NaN,NaN,NaN
2,bosanski petrovac,2005,655.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1286.0,NaN,NaN,NaN
3,buzim,2005,677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1105.0,NaN,NaN,NaN
4,cazin,2005,721.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4908.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,drvar,2024,1492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1081.0,NaN,NaN,NaN
1576,glamoc,2024,1708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,628.0,NaN,NaN,NaN
1577,kupres,2024,2080.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,979.0,NaN,NaN,NaN
1578,livno,2024,2104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4865.0,NaN,NaN,NaN


Going to save this for now

In [9]:
df.to_excel(SAVE_DIR + "merged_no_politics.xlsx", index=False)

In [10]:
total = pd.read_excel(SAVE_DIR + "merged_no_politics.xlsx")
total

,Municipality,Year,Gross Average Wage,00-Unclassified according to activities CEA 1),"A-Agriculture, forestry and fishing",Activities of households as employers; undifferentiated goods and services-producing activities of households for own use,B-Minning and quarrying,C-Manufacturing,"D-Electricity, gas, steam and air conditioning supply","E-Water supply, sewerage, waste management and remediation activities",...,O-Public administration and defence; compulsory social security,P-Education,Q-Human health and social work activities,"R-Arts, entertainment and recreation",S-Other service activities,Total,Employees,Bosniak,Croat,Serb
0,bihac,2005,824.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11185.0,NaN,NaN,NaN
1,bosanska krupa,2005,717.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2272.0,NaN,NaN,NaN
2,bosanski petrovac,2005,655.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1286.0,NaN,NaN,NaN
3,buzim,2005,677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1105.0,NaN,NaN,NaN
4,cazin,2005,721.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4908.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,drvar,2024,1492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1081.0,NaN,NaN,NaN
1576,glamoc,2024,1708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,628.0,NaN,NaN,NaN
1577,kupres,2024,2080.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,979.0,NaN,NaN,NaN
1578,livno,2024,2104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4865.0,NaN,NaN,NaN


Most of the data is available from `2012` to `2023`.

## Transform and Merge Political data

Below will be use full for processing data in the db to be presentable as a df.

In [28]:
sql_query = """
    SELECT
        m.name AS municipality_name,
        y.year,        
        SUM(cr.votes) AS total_votes,
    
        SUM(CASE 
            WHEN c.ethnicity = 'B' THEN cr.votes 
            ELSE 0 
        END) AS bosnian_votes,
        
        (SUM(CASE 
            WHEN c.ethnicity = 'B' THEN cr.votes 
            ELSE 0 
        END) * 100.0) / SUM(cr.votes) AS percentage_bosnian_votes,
    
        SUM(CASE 
            WHEN c.ethnicity = 'C' THEN cr.votes 
            ELSE 0 
        END) AS croat_votes,
        
        (SUM(CASE 
            WHEN c.ethnicity = 'C' THEN cr.votes 
            ELSE 0 
        END) * 100.0) / SUM(cr.votes) AS percentage_croat_votes
    
    FROM
        Municipalities AS m
    JOIN
        Years AS y ON m.year_id = y.year_id
    JOIN
        CandidateResults AS cr ON m.municipality_id = cr.municipality_id
    JOIN
        Candidates AS c ON cr.candidate_id = c.candidate_id
    GROUP BY
        m.name, y.year_id;
"""

conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
conn.execute('PRAGMA foreign_keys = ON')

try:
    pol_df = pd.read_sql_query(sql_query, conn)

except sqlite3.Error as e:
    print(f"An error occurred with the SQLite database: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

conn.close()

In [29]:
pol_df

,municipality_name,year,total_votes,bosnian_votes,percentage_bosnian_votes,croat_votes,percentage_croat_votes
0,banovici,2022,14137,11985,84.777534,2152,15.222466
1,banovici,2018,15082,13053,86.546877,2029,13.453123
2,banovici,2014,15567,15374,98.760198,193,1.239802
3,banovici,2010,14757,9675,65.562106,5082,34.437894
4,bihac,2022,21175,13471,63.617473,7704,36.382527
...,...,...,...,...,...,...,...
315,zepce,2010,11418,3447,30.189175,7971,69.810825
316,zivinice,2022,28001,20301,72.500982,7700,27.499018
317,zivinice,2018,27887,19560,70.140209,8327,29.859791
318,zivinice,2014,27733,25680,92.597267,2053,7.402733


In [31]:
pol_df = pol_df.drop(columns=["total_votes", "bosnian_votes", "croat_votes"])
pol_df = pol_df.rename(
    columns={
        "municipality_name" : "Municipality",
        "year" : "Year",
    }
)

In [32]:
combined = pd.merge(total, pol_df, on=["Municipality", "Year"], how="left")

In [33]:
combined

,Municipality,Year,Gross Average Wage,00-Unclassified according to activities CEA 1),"A-Agriculture, forestry and fishing",Activities of households as employers; undifferentiated goods and services-producing activities of households for own use,B-Minning and quarrying,C-Manufacturing,"D-Electricity, gas, steam and air conditioning supply","E-Water supply, sewerage, waste management and remediation activities",...,Q-Human health and social work activities,"R-Arts, entertainment and recreation",S-Other service activities,Total,Employees,Bosniak,Croat,Serb,percentage_bosnian_votes,percentage_croat_votes
0,bihac,2005,824.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11185.0,NaN,NaN,NaN,NaN,NaN
1,bosanska krupa,2005,717.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2272.0,NaN,NaN,NaN,NaN,NaN
2,bosanski petrovac,2005,655.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1286.0,NaN,NaN,NaN,NaN,NaN
3,buzim,2005,677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1105.0,NaN,NaN,NaN,NaN,NaN
4,cazin,2005,721.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4908.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,drvar,2024,1492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1081.0,NaN,NaN,NaN,NaN,NaN
1576,glamoc,2024,1708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,628.0,NaN,NaN,NaN,NaN,NaN
1577,kupres,2024,2080.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,979.0,NaN,NaN,NaN,NaN,NaN
1578,livno,2024,2104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4865.0,NaN,NaN,NaN,NaN,NaN


Going to combine this now and then clean up and choose which years later.

In [34]:
combined.to_excel(SAVE_DIR + "combined.xlsx", index=False)

In [35]:
df = pd.read_excel(SAVE_DIR + "combined.xlsx")
df

,Municipality,Year,Gross Average Wage,00-Unclassified according to activities CEA 1),"A-Agriculture, forestry and fishing",Activities of households as employers; undifferentiated goods and services-producing activities of households for own use,B-Minning and quarrying,C-Manufacturing,"D-Electricity, gas, steam and air conditioning supply","E-Water supply, sewerage, waste management and remediation activities",...,Q-Human health and social work activities,"R-Arts, entertainment and recreation",S-Other service activities,Total,Employees,Bosniak,Croat,Serb,percentage_bosnian_votes,percentage_croat_votes
0,bihac,2005,824.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11185.0,NaN,NaN,NaN,NaN,NaN
1,bosanska krupa,2005,717.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2272.0,NaN,NaN,NaN,NaN,NaN
2,bosanski petrovac,2005,655.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1286.0,NaN,NaN,NaN,NaN,NaN
3,buzim,2005,677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1105.0,NaN,NaN,NaN,NaN,NaN
4,cazin,2005,721.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4908.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,drvar,2024,1492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1081.0,NaN,NaN,NaN,NaN,NaN
1576,glamoc,2024,1708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,628.0,NaN,NaN,NaN,NaN,NaN
1577,kupres,2024,2080.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,979.0,NaN,NaN,NaN,NaN,NaN
1578,livno,2024,2104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4865.0,NaN,NaN,NaN,NaN,NaN
